In [1]:
import abc
import pandas as pd
import os
from pandas import Series, DataFrame
from configparser import SafeConfigParser
import codecs
import ast

In [2]:
import sys
import sqlite3
import pandas as pd
from pandas import Series, DataFrame

class DbGenericOperator(object):
    __metaclass__ = abc.ABCMeta
    
    @abc.abstractmethod
    def opendb(self):
        raise NotImplementedError
    
    @abc.abstractmethod
    def closedb(self):
        raise NotImplementedError

    @abc.abstractmethod
    def readtbl(self,query):
        raise NotImplementedError
    
    @abc.abstractmethod
    def writetbl(self,query):
        raise NotImplementedError      

    @abc.abstractmethod    
    def updatetbl(self):
        raise NotImplementedError
  
    @abc.abstractmethod
    def deletetbl(self,query):
        raise NotImplementedError
        
class DbSqLiteOperator(DbGenericOperator):


    def __init__(self, params):

        self.dbpath = params['dbpath']
        self.dbname = params['dbname']

    def openDb(self):
        self.conn = sqlite3.connect(self.dbpath + self.dbname, detect_types = sqlite3.PARSE_DECLTYPES)
        
    def closeDb(self):
        self.conn.close()
        
    def readTbl(self, query):
        df = pd.read_sql_query(query, self.conn)
        return df
     
    def writeTbl(self, sql, tuplas):
        cursor = self.conn.cursor()
        cursor.executemany(sql, tuplas)
        self.conn.commit()
        cursor.close()
        
    def updateTbl(self, sql, args):      
        cursor = self.conn.cursor()
        cursor.executemany(sql, args)
        self.conn.commit()
        cursor.close()
        
    def deleteTbl(self, query):
        cursor = self.conn.cursor()
        cursor.execute(query)
        self.conn.commit()

In [3]:
class DbManager(object):
    
    __metaclass__ = abc.ABCMeta
    
    @abc.abstractmethod 
    def prepare(self):
        pass
    
    @abc.abstractmethod       
    def operation(self):
        pass
  
    def display(self, paramstable):
        print('Registros de la tabla %s es %s registros ' % ((paramstable['section'], paramstable['lenght'])))
    
class LoadTbl(DbManager):
    
    def __init__(self, params):
        self.params = params
        
    def prepare(self):
        pass
    
    def operation(self):
        
        dboperator = DbSqLiteOperator(self.params)
        dboperator.openDb()
        df = dboperator.readTbl(self.params['query'])
        dboperator.closeDb()
        df = df.fillna('')
        return df

class DbDataProcess(object):
    
    def __init__(self, month):
        self.month = month
        self.parser = None
        self.section = None
        self.parameters = None
        self.dbpath = 'D:/Datos de Usuario/cleon/Documents/Capital Humano/Bases/'
        self.dbname = 'comisiones3.sqlite'
        
    def configParameters(self): # incluía parser
   

        l2 = []
        
        itemlist = ['colstoinsert', 'coldates', 'colstoupdate']
    
        for item in self.parser.options(self.section): # 
            if item in itemlist:
                l2.append(ast.literal_eval(self.parser.get(self.section,item)))      
            else:
                l2.append(self.parser.get(self.section,item))

        self.parameters = dict(zip(self.parser.options(self.section),l2))

        self.parameters['section'] = self.section
        self.parameters['dbpath'] = self.dbpath
        self.parameters['dbname'] = self.dbname
                
    
    def loadData(self, section):
        
        self.section = section
        self.configParameters()
        
        dbobj = LoadTbl(self.parameters)
        df = dbobj.operation()
        df.name = section
        
        paramstable = {'section' : self.section, 'lenght' : len(df)}
        dbobj.display(paramstable)
        
        return df
        
    def setParser(self, parser):
        self.parser = parser
        

In [4]:
from Loader import fileloader as fl
from Loader import datavalidator as dv

month = '201708'
testpath = 'D:/Datos de Usuario/cleon/Documents/Capital Humano/Data Fuente Comisiones/test/'
defaultpath = 'D:/Datos de Usuario/cleon/Documents/Capital Humano/Data Fuente Comisiones/xlsx/'


In [5]:
inifile = fl.ReadIniFile('C:/Anaconda3/MeScripts/Comisiones4/Config/myconfig.ini')
parser = inifile.readFile()

loader = fl.LoadFileProcess(month)
loader.setParser(parser)
loader.setDefaultPath(defaultpath)

In [6]:
# Importando la información

inarbruto = loader.loadFile('Inar_bruto')
jerarquia = loader.loadFile('Jerarquia')
comisionantes = loader.loadFile('Comisionantes_voz')

inidb = fl.ReadIniFile('C:/Anaconda3/MeScripts/Comisiones4/Config/mydbconfig.ini')
dbparser = inidb.readFile()

dbmanager = DbDataProcess(month)
dbmanager.setParser(dbparser)
tblempleados = dbmanager.loadData('tblEmpleados')


El tamaño de Inar_bruto es 965904 registros
El tamaño de Jerarquia es 24103 registros
El tamaño de Comisionantes_voz es 330 registros


C:\Anaconda3\lib\site-packages\pandas\io\parsers.py:1170: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


KeyError: 'query'

In [ ]:
# Armando las validaciones

valinarfl = {'inarbruto' : inarbruto, 'jerarquia' : jerarquia, 'comisionantes' : comisionantes, 'tblempleados' : tblempleados}
valinarobj = dv.ValidateInar(valinarfl)
inarvalidation = valinarobj.validation()


valmultipledatafl = {'jerarquia' : jerarquia, 'comisionantes' : comisionantes, 'tblempleados' : tblempleados}
valmultipledataobj = dv.ValidateMultipleData(valmultipledatafl)
multipledatavalidation = valmultipledataobj.validation()

#****** Exportar validaciones

xlsxfile1 = testpath + month + '_Validaciones en INAR.xlsx'
exportparams = {'xlsxfile' : xlsxfile1, 'dataframe' : inarvalidation}
valinarobj.exportValidation(exportparams)

xlsxfile2 = testpath + month + '_Otras Validaciones.xlsx'
exportparams = {'xlsxfile' : xlsxfile2, 'dataframe' : multipledatavalidation}
valmultipledataobj.exportValidation(exportparams)

In [ ]:
exportar = False
if exportar:
    df1 = inarbruto[:500000]
    df2 = inarbruto[500001:]
    df1.to_csv(testpath + month + '_herman1.csv', index = False)
    df2.to_csv(testpath + month + '_herman2.csv', index = False)